In [1]:
include(joinpath("module", "lattice.jl"));

In [2]:
include(joinpath("module", "exp_parameter.jl"));

In [3]:
include(joinpath("module", "par_exp_short_FISTA.jl"));

In [4]:
include(joinpath("module", "par_exp_Long_FISTA.jl"));

In [5]:
using BenchmarkTools
using .LatticeNetwork
using .Parameter
using .par_Short
using .par_Long
using LinearAlgebra
using Random
using Profile
using ProfileView
using StatProfilerHTML
using Printf;

In [6]:
# 初期パラメータ設定
E = 5
Col = 2
K = Col * Col
M = 1.0
N = 1.0
Scaling = 10.0 / Col
alter_T_num = 0.5

# 格子ネットワークを生成
city_network = make_lattice(Col)

# 通勤費用パラメータ
t = 0.1
# 距離抵抗パラメータ
tau = 0.5
# 総土地供給量と平均床面積
S_total = 100
S_bar = S_total / K

# 座標データの生成
Coordinate_Data = [ 
    (city_network["node_dic"][string(i)]["x_pos"] * Scaling,
     city_network["node_dic"][string(i)]["y_pos"] * Scaling) 
    for i in 0:(K - 1)
]

# 距離行列の作成
distance_matrix = [sqrt(sum((Coordinate_Data[i] .- Coordinate_Data[j]).^2))
                   for i in 1:K, j in 1:K]

# モデルパラメータ設定
theta_firm = 1.0
theta_house = 1.0
alpha_1 = 0.4
alpha_2 = 0.4
beta_1 = 0.4
beta_2 = 0.4
# L = 1.0
# eta = 5.0
L = 0.2
eta = 1.2
p_proj = 1e-3
RW_ini = 1.0
RW_proj = 1e-3
err_short = 1e-5
err_long = 1e-3
obj_corr = 1.0
m_per = M / K
m0 = fill(m_per, K)
n0 = fill(N / (K * K), (K, K))
method = "FISTA"
dic = "Scaling=30"

K = Col * Col
T = max.(Scaling * t * alter_T_num, t * distance_matrix)
D = exp.(-tau * distance_matrix)

power_S_H_R = (1 - alpha_2) / (1 - alpha_1 - alpha_2)
power_S_H_W = alpha_2 / (1 - alpha_1 - alpha_2)
power_L_H_R = alpha_1 / (1 - alpha_1 - alpha_2)
power_L_H_W = (1 - alpha_1) / (1 - alpha_1 - alpha_2)

power_S_F_R = (1 - beta_2) / (1 - beta_1 - beta_2)
power_S_F_W = beta_2 / (1 - beta_1 -beta_2)
power_L_F_R = beta_1 / (1 - beta_1 - beta_2)
power_L_F_W = (1 - beta_1) / (1 - beta_1 - beta_2)

T_power = (1 ./ exp.(T)) .^ (1 / (1 - alpha_1 - alpha_2))
alpha_S_R = alpha_1 ^ power_S_H_R
alpha_S_W = alpha_2 ^ power_S_H_W
alpha_L_R = alpha_1 ^ power_L_H_R
alpha_L_W = alpha_2 ^ power_L_H_W

beta_S_R = beta_1 ^ power_S_F_R
beta_S_W = beta_2 ^ power_S_F_W
beta_L_R = beta_1 ^ power_L_F_R
beta_L_W = beta_2 ^ power_L_F_W

RW_S_H = zeros(K, K)
RW_L_H = zeros(K, K)

T_power_v = (1 - alpha_1 - alpha_2) * T_power
pi_beta = 1 - beta_1 - beta_2

one_K = ones(K)
one_2K = ones(2 * K)

R_ini_list = ones(K)
W_ini_list = ones(K)
RW_ini_list = ones(2 * K)

short_itr = 1000
long_itr = 1000

# パラメータオブジェクトの生成
prm = st_Parameter(
    Col, K, tau, t, Scaling, S_total, S_bar,
    theta_firm, theta_house, E, RW_proj,
    alpha_1, alpha_2, beta_1, beta_2, alter_T_num,
    M, N, distance_matrix, T, D,
    power_S_H_R, power_S_H_W, power_L_H_R, power_L_H_W,
    power_S_F_R, power_S_F_W, power_L_F_R, power_L_F_W, T_power,
    alpha_S_R, alpha_S_W, alpha_L_R, alpha_L_W,
    beta_S_R, beta_S_W, beta_L_R, beta_L_W,
    T_power_v, pi_beta, one_K, one_2K, RW_S_H, RW_L_H
)

algprm = st_Algo_Parameter(L, eta, p_proj);

In [8]:
@time R_hist, W_hist, g, obj_rel_list = short_solve(prm, algprm, RW_ini, m0, n0, err_short, short_itr);

iteration:1
btra_itr:27
L:27.474110393491852
iteration:2
btra_itr:16
L:507.9530538901155
iteration:3
btra_itr:11
L:3774.1337095688846
Adaptive_on
iteration:4
btra_itr:0
L:3774.1337095688846
iteration:5
btra_itr:0
L:3774.1337095688846
iteration:6
btra_itr:0
L:3774.1337095688846
iteration:7
btra_itr:0
L:3774.1337095688846
iteration:8
btra_itr:0
L:3774.1337095688846
Adaptive_on
iteration:9
btra_itr:0
L:3774.1337095688846
iteration:10
btra_itr:0
L:3774.1337095688846
iteration:11
btra_itr:0
L:3774.1337095688846
iteration:12
btra_itr:0
L:3774.1337095688846
iteration:13
btra_itr:0
L:3774.1337095688846
iteration:14
btra_itr:0
L:3774.1337095688846
iteration:15
btra_itr:0
L:3774.1337095688846
iteration:16
btra_itr:0
L:3774.1337095688846
iteration:17
btra_itr:0
L:3774.1337095688846
iteration:18
btra_itr:0
L:3774.1337095688846
iteration:19
btra_itr:0
L:3774.1337095688846
iteration:20
btra_itr:0
L:3774.1337095688846
iteration:21
btra_itr:0
L:3774.1337095688846
iteration:22
btra_itr:0
L:3774.1337095

In [9]:
R_hist

4-element Vector{Float64}:
 0.0491408106944193
 0.0491408106944193
 0.0491408106944193
 0.0491408106944193

In [10]:
W_hist

4-element Vector{Float64}:
 0.9830748447562767
 0.9830748447562767
 0.9830748447562767
 0.9830748447562767

In [15]:
dR = zeros(K)
dW = zeros(K)

L_bar = backtracking(prm, algprm, dR::Vector{Float64}, dW::Vector{Float64}, RW_ini_list::Vector{Float64}, L::Float64, m0::Vector{Float64}, n0::Matrix{Float64})

L_bar

L_H_W:[0.06399999999999999, 0.06399999999999999, 0.06399999999999999, 0.06399999999999999]
L_F:[0.010239999999999994, 0.010239999999999994, 0.010239999999999994, 0.010239999999999994]


27.474110393491852

In [13]:
Z = Z_SD(prm, RW_ini_list::Vector{Float64}, m0::Vector{Float64}, n0::Matrix{Float64})

Z

105.00573416702365

In [11]:
pi_reuslt = pi_noex(prm, R_ini_list::Vector{Float64}, W_ini_list::Vector{Float64}, m0::Vector{Float64})

println(pi_reuslt)

[0.005119999999999995, 0.005119999999999995, 0.005119999999999995, 0.005119999999999995]


In [10]:
dR = zeros(K)
dW = zeros(K)

dRdW = short_dual_df(prm, dR::Vector{Float64}, dW::Vector{Float64}, RW_ini_list::Vector{Float64}, m0::Vector{Float64}, n0::Matrix{Float64})

println(dRdW)

L_H_W:[0.06399999999999999, 0.06399999999999999, 0.06399999999999999, 0.06399999999999999]
L_F:[0.010239999999999994, 0.010239999999999994, 0.010239999999999994, 0.010239999999999994]
[24.99713291648818, 24.99713291648818, 24.99713291648818, 24.99713291648818, 1.2471329164881797, 1.2471329164881797, 1.2471329164881797, 1.2471329164881797]


In [ ]:
dR = zeros(K)
dW = zeros(K)

for i in 1:100
    println("=======================================")
    println("Before allocation: ", Base.gc_bytes(), " bytes")
    short_dual_df(prm, dR::Vector{Float64}, dW::Vector{Float64}, RW_ini_list::Vector{Float64}, m0::Vector{Float64}, n0::Matrix{Float64})
    println("After allocation: ", Base.gc_bytes(), " bytes")

    GC.gc()
    println("After GC: ", Base.gc_bytes(), " bytes")
end

In [11]:
itr

5

In [38]:
dR = zeros(prm.K)
dW = zeros(prm.K)
equ_R, equ_W = equilibrium(prm, dR, dW, R_hist, W_hist, m0, n0)

(1.4350006042101972e-5, -0.0012879240188393218)

In [ ]:
par_long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr);

In [ ]:
@time par_long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr);

Long_max_value:79Lyapunov:-3.0371067168744048e-5
100168.706802 seconds (114.60 M allocations: 377.067 TiB, 16.51% gc time)


In [10]:
@profilehtml long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr);
# @profile long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr);

Long_max_value:73Lyapunov:-3.581404741304084e-5


[ Info: Wrote profiling output to file://C:\Users\kimura\qse_nume\FISTA\exp_jul\statprof/index.html .


In [7]:
@profile begin
    long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr)
end

StatProfilerHTML.print()

Long_max_value:73Lyapunov:-3.581404741304084e-5


In [8]:
@profile long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr)
ProfileView.view();

Long_max_value:73Lyapunov:-3.581404741304084e-5
.\compiler\abstractinterpretation.jl:2624, MethodInstance for Core.Compiler.abstract_eval_statement(::Core.Compiler.NativeInterpreter, ::Any, ::Vector{Core.Compiler.VarState}, ::Core.Compiler.InferenceState)
.\compiler\abstractinterpretation.jl:2624, MethodInstance for Core.Compiler.abstract_eval_statement(::Core.Compiler.NativeInterpreter, ::Any, ::Vector{Core.Compiler.VarState}, ::Core.Compiler.InferenceState)


In [10]:
@time long_solve(prm, algprm, m0, n0, RW_ini, err_short, err_long, obj_corr, long_itr);

Long_max_value:73Lyapunov:-3.581404741304084e-5
  9.569168 seconds (4.70 M allocations: 41.532 GiB, 10.01% gc time)


In [4]:
mn0 = bond(prm, m0, n0)
Z_LP(prm, mn0, RW_ini_list)

0.06456578673026336

In [5]:
mn0 = bond(prm, m0, n0)
logit(prm, R_ini_list, W_ini_list, m0)

10100-element Vector{Float64}:
 0.0863572793382453
 0.09017922077410445
 0.09327306725725167
 0.09544328950209136
 0.0965607877591745
 0.0965607877591745
 0.09544328950209137
 0.09327306725725167
 0.0901792207741045
 0.0863572793382453
 0.0901792207741045
 0.09465113039366298
 0.09827400744428184
 ⋮
 0.0009998528966590169
 0.0010004852390266576
 0.000999717174345137
 0.0009997171847735394
 0.000999717207024991
 0.0009997172600801705
 0.000999717407033188
 0.0009997179111308205
 0.0009997203209475786
 0.0009997411548424613
 0.0010004852390266576
 0.0010245906032452037

In [5]:
short_solve(prm, algprm, RW_ini, m0, n0, err_short, short_itr)

(Any[0.05428658206079239, 0.054383244584201205, 0.054387803327958124, 0.05438861068257685, 0.054388812984597946, 0.054388812984598155, 0.05438861068257678, 0.054387803327958124, 0.0543832445842015, 0.05428658206079236  …  0.05428658206079239, 0.0543832445842012, 0.054387803327958, 0.05438861068257674, 0.05438881298459795, 0.054388812984598155, 0.054388610682576784, 0.054387803327957916, 0.05438324458420116, 0.05428658206079238], Any[0.9772988630006281, 0.9790629119524241, 0.9791314639813392, 0.9791444767542825, 0.9791478603783514, 0.9791478603783456, 0.9791444767542825, 0.9791314639813392, 0.9790629119524148, 0.9772988630006281  …  0.9772988630006281, 0.9790629119524241, 0.9791314639813434, 0.9791444767542825, 0.9791478603783514, 0.9791478603783456, 0.9791444767542825, 0.9791314639813434, 0.9790629119524241, 0.9772988630006281], 494, Any[])

In [16]:
@time short_solve(prm, algprm, RW_ini, m0, n0, err_short, short_itr)

  0.585736 seconds (12.13 k allocations: 1.852 GiB, 43.64% gc time)


(Any[0.15307886857387734, 0.1538724237301507, 0.1539013263555799, 0.15390728856488983, 0.15390914476850265, 0.15390988710261921, 0.15391023285593397, 0.15391040805445516, 0.15391049742057994, 0.1539105356336611  …  0.15391053563366108, 0.15391049742057997, 0.15391040805445524, 0.15391023285593397, 0.1539098871026191, 0.15390914476850256, 0.15390728856488983, 0.15390132635557988, 0.15387242373015061, 0.15307886857387734], Any[0.6890856436736664, 0.6926250752599873, 0.6927600248908615, 0.6927872056742209, 0.6927955519254143, 0.6927988847862605, 0.6928004376441361, 0.6928012246115425, 0.6928016260209724, 0.6928017976545034  …  0.6928017976545034, 0.6928016260209723, 0.692801224611542, 0.6928004376441362, 0.6927988847862614, 0.692795551925415, 0.6927872056742207, 0.6927600248908616, 0.6926250752599877, 0.6890856436736664], 60, Any[])

In [4]:
cal_pi(prm, R_ini_list, W_ini_list, m0)

25-element Vector{Float64}:
 1.3841881620188725
 1.5036302763215232
 1.5446821382634097
 1.5036302763215237
 1.3841881620188727
 1.5036302763215235
 1.6430150832087074
 1.6907828882908504
 1.6430150832087074
 1.5036302763215232
 1.5446821382634093
 1.6907828882908502
 1.740909786357349
 1.6907828882908507
 1.5446821382634095
 1.5036302763215235
 1.6430150832087076
 1.6907828882908504
 1.6430150832087076
 1.5036302763215235
 1.3841881620188727
 1.5036302763215232
 1.5446821382634097
 1.5036302763215235
 1.3841881620188727

In [6]:
pi_noex(prm, R_ini_list, W_ini_list, m0)

25-element Vector{Float64}:
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995
 0.005119999999999995

In [ ]:
v(prm, R_ini_list, W_ini_list)

In [8]:
Z_SD(prm, RW_ini_list, m0, n0)

912.5133732018587

In [10]:
short_dual_df(prm, RW_ini_list, m0, n0)

50-element Vector{Float64}:
 35.998932089977565
 35.998930559354456
 35.998930499326484
 35.998930559354456
 35.998932089977565
 35.998930559354456
 35.99892880438462
 35.998928725812966
 35.99892880438462
 35.998930559354456
 35.998930499326484
 35.998928725812966
 35.99892864344036
  ⋮
  0.4989287258129678
  0.4989304993264839
  0.49893055935445507
  0.4989288043846178
  0.49892872581296777
  0.4989288043846178
  0.49893055935445507
  0.4989320899775639
  0.49893055935445507
  0.49893049932648387
  0.49893055935445496
  0.49893208997756394

In [4]:
short_solve

short_solve (generic function with 1 method)